# Exercise 1: Graph generation using `netin`
## Overview
In this workshop, we will focus on generating synthetic network data using the `netin` package, which is designed to simulate and analyze networks with a specific emphasis on the existence of a minority group, connecting through common link formation mechanisms, and the measurement of network inequalities.

This exercise focuses on the simulation of existing network models with varying parameter configurations and the introduction of the data structures introduced by the package.

Check the [documentation](https://cshvienna.github.io/NetworkInequalities/alpha/) to learn about the package details.
Here, we will focus on the follow package modules:
- `models`: Contains the network models to simulate (e.g., `PAHModel`)
- `graphs`: Internal graph classes and vectors of node data.

<img src="images/class_structure.png" alt="Class Structure" style="width: 900px;"/>

### Key Concepts

1. **Network Generation**: Definition and simulation of various network models and potential parameters, such as a homophilic or heterophilic `PAHModel`.
2. **Data Structures and Model Extension**: Handling and manipulation of important data structures provided by the `netin` package. How to inject your own code to analyze or change the model behavior at runtime.
3. **The Effects of Homophily**: A use-case to study the effects of homophily on the segregation of the network.

### Task
1. Define and simulate a range of existing undirected and directed network models.
2. Retrieve and analyze node attributes like the minority/majority group-assignment or their degrees.
3. Study the effect of homophily on network segregation through the simulation and analysis of three models.
4. Inject your own code to track the growth of in- and out-group links as a function of the simulation time. 

## Dependencies

In [ ]:
### If running this on Google Colab, run the following lines:

import os
!pip install netin==2.0.0a1
!pip install networkx==3.2.1
os.kill(os.getpid(), 9)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

from netin.models import\
    BarabasiAlbertModel, HomophilyModel, PAHModel,\
    PATCHModel, CompoundLFM,\
    DPAModel, DPAHModel
from netin.graphs import\
    Graph,\
    NodeVector, CategoricalNodeVector
from netin.utils import CLASS_ATTRIBUTE, Event
from netin import viz

## Undirected graphs
We start with the simulation of an undirected `HomophilyModel`.
This model connects a new node to the existing nodes biased by their group assignment.
Homophily values of `h>0.5` indicate homophily, meaning that nodes prefer to connect to their own group.
Heterophily is specified by `h<0.5`.
Nodes tend to connect to the other group.

In [3]:
N = 200 # Number of total nodes
m = 3 # Number of new links per joining node
f_m = 0.3 # Fraction of minority nodes
h = 0.8 # Homophilic

h_model = HomophilyModel(N=200, m=3, f_m=0.3, h_m=h, h_M=h)
h_graph = h_model.simulate()

**Task**: Instantiate the following network models with arbitrary parameters and simulate them (see the [model documentation page](https://cshvienna.github.io/NetworkInequalities/alpha/models/index.html)):
1. `BarabasiAlbertModel`: A popular network model based on preferential attachment (PA).
2. `PAHModel`: An extension to the `BarabasiAlbertModel` which adds homophily (H).
3. `PATCHModel` \[Advanced\]: Another extension including triadic closure (TC).
   - Triadic closure creates two types of links. Those that are limited to friends of friends and those that can target every other node. Each selection can be unbiased (uniform) selection, by PA or PA+H. The choice which mechanism to use can be specified by passing a `CompoundLFM` in the constructor of `PATCHModel`. For instance, passing `CompoundLFM.PAH` for both link types will bias the target node selection by preferential attachment and homophily.

### Solution
We start with general parameters and the `BarabasiAlbertModel`.

In [4]:
ba_model = BarabasiAlbertModel(N=N, m=m)
ba_graph = ba_model.simulate()

Add symmetric homophily, define and simulate the `PAHModel`.

In [5]:
h=0.5
pah_model = PAHModel(N=N, m=m, f_m=f_m, h_M=h, h_m=h)
pah_graph = pah_model.simulate()

Choose link formation mechanism for both global and local links and triadic closure probability.
Define and simulate `PATCHModel`.

In [6]:
ptc = 0.8
lfm = CompoundLFM.PAH
patch_model = PATCHModel(
    N=N, m=m, f_m=f_m,
    p_tc=ptc,
    lfm_global=lfm, lfm_local=lfm,
    lfm_params={"h_M":h, "h_m":h})
patch_graph = patch_model.simulate()

## Directed graphs
Directed networks consider the directionality of links.
A link `a->b` does not imply the presence of `b->a`.
Nodes can thus have two link types, incoming or outgoing links.

Instead of the number of links per each new node, the directed graph models parameterize the [density](https://en.wikipedia.org/wiki/Dense_graph) of the final network by the parameter `d`.
Additionally, we need to define activity levels for outgoing links for both the minority and majority group as `plo_m/M`.

**Task**: Create and simulate the two models `DPAModel` and `DPAHModel`.
_Hint_: `plo_m/M` should be larger than zero.

In [ ]:
d = 0.01
plo = 2.0

### Solution

In [7]:
dpa_model = DPAModel(N=N, f_m=f_m, d=d, plo_M=plo, plo_m=plo)
dpa_graph = dpa_model.simulate()

In [8]:
dpa_model = DPAHModel(N=N, f_m=f_m, d=d, plo_M=plo, plo_m=plo, h_M=h, h_m=h)
dpa_graph = dpa_model.simulate()

## Data structures
The result of a simulation is a `Graph` or `DiGraph` (see [documentation](https://cshvienna.github.io/NetworkInequalities/alpha/graphs.html)), depending on whether a undirected or directed model was used.

In [9]:
pah_model = PAHModel(N=N, m=m, f_m=f_m, h_M=h, h_m=h, seed=123)
pah_graph = pah_model.simulate() # alternatively: pah_model.graph

In [ ]:
type(pah_graph)

These graphs can also be created manually.

In [ ]:
custom_graph = Graph()

custom_graph.add_node(1)
custom_graph.add_node(2)
custom_graph.add_edge(1, 2)

custom_graph.has_edge(1, 2)

_Note_ that the `netin` is not an advanced graph manipulation package.
However, it offers an interface to import or export graphs from/to [NetworkX](https://networkx.org/documentation/stable/index.html), a popular Python networks library.

In [ ]:
nx_pah_graph = pah_graph.to_nxgraph() # This creates a copy of the graph
nx_pah_graph

The reversed way is also possible:

In [ ]:
nx_graph = nx.Graph()
nx_graph.add_nodes_from(list("abcd"))
nx_graph.add_edge("a", "b")
nx_graph

In [ ]:
node_ids, netin_graph = Graph.from_nxgraph(nx_graph)
netin_graph

Importing from a NetworkX graph returns a mapping of the node IDs (`node_ids`), because NetworkX allows custom node labels while `netin` stores nodes as integers.

`node_ids` is a `NodeVector`, a wrapper for numpy arrays that assigns a value to each node.
In this vector, the value is the node id of the original NetworkX graph.

In [ ]:
node_ids[0] # The original node id of node 0

`NodeVectors` are used to compute connection probabilities and store the minority class membership.
Most of the predefined model classes (such as `PAHModel`) assign a `minority` class attribute to the graph.
This vector describes the minority group membership for each node.

In [ ]:
CLASS_ATTRIBUTE

The minority node labels are assigned to the graph that was created during the simulation.

In [ ]:
nodes_minority = pah_graph.get_node_class(CLASS_ATTRIBUTE)
nodes_minority

In [ ]:
nodes_minority.vals() # Get the original `np.ndarray`

There are three abstraction levels of `NodeVector`s based on the type of the stored values:
1. `NodeVector` holds any type that can be stored in a `np.ndarray` (general purpose).
2. `CategoricalNodeVector` stores integers and provides functions to map these integers to labels.
3. `BinaryClassNodeVector` assigns a binary value to each node, providing additional methods to create and query minority assignments.

These classes inherit from each other: `NodeVector -> CategoricalNodeVector -> BinaryClassNodeVector`.

In [ ]:
type(nodes_minority)

In [ ]:
isinstance(nodes_minority, NodeVector)

In [ ]:
isinstance(nodes_minority, CategoricalNodeVector)

In [ ]:
nodes_minority.get_class_values() # A function of CategoricalNodeVector to translate the values to class labels

In [ ]:
nodes_minority.get_n_minority() # A function of BinaryClassNodeVector to get the number of minority nodes

`NodeVector` can be used with numpy functions:


In [ ]:
np.sum(nodes_minority)

**Tasks**:
1. Create a DPAH model with a minority fraction of `f_m=0.1`. From the simulated network, retrieve the minority assignment and compute the actual fraction of nodes that were assigned to the minority class.
   - _Hint_: Minority status is encoded as `1` and majority as `0`. Use numpy functions to quickly assess the fraction of minority nodes in the `NodeVector`. You can try different randomization seeds by providing varying integers to the `DPAHModel` constructor.
2. Report the average and standard deviation of degrees
    - _Hint_: Use `DiGraph.degrees()` and numpy functions.

### Solution

In [ ]:
dpah_model = DPAHModel(N=N, f_m=0.1, d=d, plo_M=plo, plo_m=plo, h_M=h, h_m=h, seed=123)
dpah_graph = dpah_model.simulate()
minority_nodes = dpah_graph.get_node_class(CLASS_ATTRIBUTE)
np.mean(minority_nodes)

In [ ]:
degrees = dpah_graph.degrees()
degrees

In [ ]:
print(f"Mean degree: {np.mean(degrees)} +- {np.std(degrees)}")

## The effect of homophily
One of the most common link formation mechanisms is homophily, the tendency of nodes to connect to other nodes with similar attributes.
In social contexts, it can lead to network segregation with regards to the respective attribute.
`netin` implements several models which implement homophily.

**Tasks**:
1. Create three `PAHModel`s
   - Make sure all of them have the same number of nodes `N`, initial links per node `m`, fraction of minority `f_m`, and random seed `seed`.
   - Use three different homophily values of $h \in \{0.2, 0.5, 0.8\}$
   - Save the models in a list called `l_models`
2. Plot the networks using `viz.plot_graph`

### Solution

In [28]:
t_homophily = (0.2, 0.5, 0.8)
l_models = [
    PAHModel(N=N, m=m, f_m=f_m, h_M=h, h_m=h) for h in t_homophily
]
for model in l_models:
    _ = model.simulate()

In [ ]:
viz.plot_graph(l_models)

It's typically difficult to infer network characteristics simply from visualizing the network.
Let's create and compare a simple metric to evaluate the network segregation.

**Task**: Compute and visualize the EI-index for each model (defined below).

$$
EI = \frac{e_{out} - e_{in}}{e_{out} + e_{in}}
$$

The $EI$ goes to -1 for segregated networks and +1 for heterophilic networks, with
- $e_{out}$: Number of out-group links (linking minority and majority group nodes)
- $e_{in}$: Number of in-group links (linking between nodes of the same group, either within minority or within the majority group)

In [30]:
def compute_ei_index(graph: Graph) -> float:
    """
    Compute the EI index of a graph.

    Parameters
    ----------
    graph : Graph
        The graph to compute the EI index of.

    Returns
    -------
    float
        The EI index of the graph.
    """
    e_in, e_out = 0, 0
    minority_nodes = graph.get_node_class(CLASS_ATTRIBUTE)
    for source, target in graph.edges():
        if minority_nodes[source] == minority_nodes[target]:
            e_in += 1
        else:
            e_out += 1
    return (e_out - e_in) / (e_out + e_in)

### Solution

In [31]:
l_ei = []
for model in l_models:
    l_ei.append(compute_ei_index(model.graph))

In [ ]:
plt.plot(range(3), l_ei, marker="s")
plt.axhline(0, color="black", linestyle="--")
plt.xticks(range(3), t_homophily)
plt.xlabel("Homophily $h$")
plt.ylabel("EI")

**Task**: \[Advanced\] Why is the EI-index for neutral homophily below the neutral value of zero?

### Solution
The main contributor are the varying group sizes. 
We are more likely to observe in-group links when one group is larger than 50% of the total population.

In [ ]:
_l_ei = []
for seed in range(100):
    _pah_model = PAHModel(N=N, m=m, f_m=0.5, h_M=0.5, h_m=0.5, seed=seed)
    _pah_graph = _pah_model.simulate()
    _l_ei.append(compute_ei_index(_pah_graph))
print(f"EI index: {np.mean(_l_ei)} +- {np.std(_l_ei)}")

## Model extensions
Various steps of analysis require modification of the existing models.
For instance, you may want to create custom models that only slightly change the pre-defined simulation logic or retrieve additional analytics about the simulation process.
The package provides is highly modular and provides interfaces for these use cases.

### Event handling

Various steps of the simulation trigger events, which we can use to inject our own code.
Let's implement a simple clock that measures how long the simulation takes.

In [ ]:
time_taken = None

def set_time(time:float):
    """Sets the initial starting time of the simulation.
    """
    global time_taken
    time_taken = time

def subtract_time(time:float):
    """Subtracts the final time after the simulation has ended.
    """
    global time_taken
    time_taken = time - time_taken

dpah_model = DPAHModel(
    N=N, f_m=f_m, d=d, plo_M=plo, plo_m=plo, h_M=h, h_m=h)

# Register event handlers that will be called when the specified event occurs
dpah_model.register_event_handler(
    Event.SIMULATION_START, # The event is triggered at the start of the simulation
    lambda: set_time(time.time())) # The function to be called when the event occurs
dpah_model.register_event_handler(
    Event.SIMULATION_START,
    lambda: subtract_time(time.time()))

dpah_model.simulate()

print(f"Time taken: {time_taken} seconds")


**Task**: Plot the number of in-group and out-group links as a function of the simulation time.
- Consider the `Event.ADD_LINK_AFTER` event triggered by the graph class and the `initialize_simulation` function of the model classes to instantiate the internal graph.

In [35]:

pah_model = PAHModel(N=N, m=m, f_m=f_m, h_M=0.8, h_m=0.8, seed=1)
pah_model.initialize_simulation() # Initializes the internal graph and node attributes

pah_graph = pah_model.graph
minority_nodes = pah_graph.get_node_class(CLASS_ATTRIBUTE)

### Solution

In [36]:
cnt_out, cnt_in = [0], [0]
def count_edge_type(source: int, target: int):
    global cnt_out, cnt_in, minority_nodes
    cnt_out.append(0)
    cnt_in.append(0)
    if minority_nodes[source] == minority_nodes[target]:
        cnt_in[-1] = 1
    else:
        cnt_out[-1] = 1

pah_graph.register_event_handler(
    Event.LINK_ADD_AFTER,
    lambda source, target: count_edge_type(source, target))

_ = pah_model.simulate()

In [ ]:
plt.step(
    range(len(cnt_in)),
    np.cumsum(cnt_in),
    label="Intra-group edges")
plt.step(
    range(len(cnt_out)),
    np.cumsum(cnt_out),
    label="Inter-group edges")

_=plt.legend()
_=plt.xlabel("Simulation time")
_=plt.ylabel("Number of edges")

## Alternative network libraries
There exist other software packages which are focused on the efficient analysis and generation of networks.
Here, we provide a list of common libraries for R and Python.

### Python libraries
- [NetworkX](https://networkx.github.io/):
  A widely-used library for the creation, manipulation, and study of complex networks. It supports random graph generation and network visualization. The `netin` package provides interfaces to NetworkX.Supports many functions for varying graph types, large community and support

- [Graph-tool](https://graph-tool.skewed.de/):
  A high-performance library focused on the efficient manipulation and generation of complex networks. Strong support for stochastic block modelling, its simulation and inference.

- [igraph](https://igraph.org/python/) (Python binding):
  Focuses on performance with large-scale graph structures and offers various algorithms and network generation methods.
  Very efficient for large networks.


### R libraries
- [igraph](https://igraph.org/r/) (R version):
    Description: Same as Python’s igraph but used within the R ecosystem.

- [statnet](https://statnet.org/):
    A suite of R packages that offer tools for analyzing social networks, including network models and visualization with a strong support for exponential random graph modelling.